Imports

In [2]:
# my imports
import requests
from lxml import html
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
import urllib

Code Fragements (Skip down)


In [ ]:
# session = requests.Session()
# page = requests.get('https://www.lineups.com/nfl/player-stats/matt-ryan')
# # print(page)
# soup = BeautifulSoup(page, 'lxml')
# print(soup)

# from lxml.html.clean import clean_html
# from selenium import webdriver
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.keys import Keys

# # the location of a webdriver on my computer
# # this thing is probably what I need if I want to extract from complex websites where forms, and buttons, and parameters
# # need to be changed on the website.
# driver = webdriver.Chrome('C:/NEW PROGRAMS/chromedriver_win32/chromedriver.exe')

# driver.get("https://www.lineups.com/nfl/roster/new-england-patriots")
# # driver.get("https://www.lineups.com/nfl/player-stats/Matt-Ryan")

#     time.sleep()

#     filename = "nfl_ne_roster.html"
#     soupString = open(filename, "r").read()
#     soup = BeautifulSoup(soupString, "html.parser")

# pos_df = pd.DataFrame(position_data)
# pos_df.fillna("Rookie", inplace=True)
# pos_df.columns = position_headers

# # filename = position + '-stats.xlsx'.xlsx
# print(filename)
# pos_df.to_excel(filename)

# position_data
# for each in position_data:
#     if len(each)>9:
#         print (each)
# print(df.loc[df['Name'] == 'John Franklin'])

# # with open(filename, "w") as file:
# #     file.write(str(soup))

# soupString = open(filename, "r").read()

# # turn back into BeautifulSoup
# soup = BeautifulSoup(soupString, "html.parser")
# pos_unique = list(df['Pos'].unique())
# print(type(pos_unique[14]))
# pos_df = df['Pos'] == 'OL'
# pos_df = df[pos_df]
# print(len(pos_df))
# print(pos_df)


Input the text file nflTeams.txt that is a list of all 32 NFL teams

Then, format the textfile into a list of properly formed URL endings

In [168]:
file = open("nflTeams.txt", "r")
nflTeams = file.read().splitlines()

for i, team in enumerate(nflTeams):
    nflTeams[i] = nflTeams[i].replace(" ", "-").lower()

print(nflTeams[:5])

['arizona-cardinals', 'atlanta-falcons', 'baltimore-ravens', 'buffalo-bills', 'carolina-panthers']


Get roster data from all NFL teams: https://www.lineups.com/nfl/roster/

Then export it to an excel file

In [80]:
base_url = "https://www.lineups.com/nfl/roster/"
teamData = []

for team in nflTeams:
    pass
    print(team)
    
    url = base_url + team
    page = requests.get(url).text
    soup = BeautifulSoup(page, 'lxml')
    
    playerRosterInfoData = soup.find('tbody')
    t_content_rows = playerRosterInfoData.findAll('tr',{'class':'t-content'})
    
    playerData = []

    # for each player
    for i,each_player in enumerate(t_content_rows):
    #     get all the column info for each found in the @td tag
        player_row = each_player.findAll('td')

        tempData = []
        tempData.append(team)

    #     each td contains info for the dataframe
        for i,col in enumerate(player_row):

    #       the players name at indices 1 needs to be formated correctly
            if (i!=1):
    #             print(col.text.strip())

    #             turn string digits into digits
                if(col.text.strip().isdigit()):
                    tempData.append(int(col.text.strip()))
                else:
                    tempData.append(col.text.strip())
            else:
                playerName_re = re.compile('\S+\s\S+') 
                tempData.append(playerName_re.match(col.text.strip()).group())
    #             print(playerName_re.match(col.text.strip()).group())

        playerData.append(tempData)
    teamData = teamData + playerData

    # wait request

arizona-cardinals
atlanta-falcons
baltimore-ravens
buffalo-bills
carolina-panthers
chicago-bears
cincinnati-bengals
cleveland-browns
dallas-cowboys
denver-broncos
detroit-lions
green-bay-packers
houston-texans
indianapolis-colts
jacksonville-jaguars
kansas-city-chiefs
los-angeles-rams
miami-dolphins
minnesota-vikings
new-england-patriots
new-orleans-saints
new-york-giants
new-york-jets
oakland-raiders
philadelphia-eagles
pittsburgh-steelers
los-angeles-chargers
san-francisco-49ers
seattle-seahawks
tampa-bay-buccaneers
tennessee-titans
washington-redskins


Total NFL players

In [81]:
len(teamData)

2835

Get the headers for the Roster Data

And print

In [82]:
# headers
headersInfo = soup.find('thead')
headerCols = headersInfo.findAll('a')
headerColList = []
headerColList.append("Team")
for i,headerCol in enumerate(headerCols):
    headerColList.append(headerCols[i].text.strip())
print(headerColList)

['Team', 'Pos', 'Name', 'Number', 'Rating', 'Ranking', 'Depth', 'Height', 'Weight', 'Age', 'Birthday', 'Exp.', 'Drafted', 'Draft Round', 'Draft Pick', 'College']


Create dataframe of data and export to excel file named:

    'nfl_rosters.xlsx'

In [31]:
# df = pd.DataFrame(teamData, columns = headerColList)
# df.to_excel('nfl_rosters.xlsx')

<h2>Getting the Positional Stats of Each Player</h2>

Import the made excel sheet from previous code into notebook

(So you don't have to rescrape every time)

In [181]:
filename = "nfl_rosters.xlsx"
df = pd.read_excel(filename)
# print(df.tail)


AttributeError: 'Series' object has no attribute 'isNaN'

Check all the positions that I am making a separate table for

In [191]:
pos_unique = list(df['Pos'].unique())
print(pos_unique)


['C', 'CB', 'DB', 'DE', 'DL', 'DT', 'FB', 'FS', 'G', 'ILB', 'K', 'LB', 'LS', 'NT', 'OL', 'OLB', 'OT', 'P', 'QB', 'RB', 'S', 'SS', 'TE', 'WR', nan]


Get all the player stats and export it to an excel file

In [166]:
pos_unique = list(df['Pos'].unique())

if 'nan' in pos_unique:
    pos_unique.remove('nan')

base_url = "https://www.lineups.com/nfl/player-stats/"


for i, position in enumerate(pos_unique[15:16]):
    print(position+ '++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++')
    pos_df = df['Pos'] == position
    pos_df = df[pos_df]
    
    # get headers for the position
    position_headers = []
    count = 0
    while(True):
        player = pos_df['Name'].iloc[count]
        url = base_url + player.replace(" " ,"-").replace("'" , '').lower()
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        pos_info = soup.find("thead")
        
        tempData = []
        tempData.append('Name')
        if (pos_info != None):   
            pos_th = pos_info.findAll('th')

            for th in pos_th:
                tempData.append(th.text.strip())
            position_headers = tempData
            break        
        count += 1
        
        if count == len(pos_df):
            position_headers = tempData
            break

    # get stats
    position_data = []
    
    for player in pos_df['Name']:
        print(player)
        tempData = []
        tempData.append(player)
        
        url = base_url + player.replace(" " ,"-").replace("'" , '').lower()
        html = urllib.request.urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')
        pos_info = soup.find("tbody")
        
        if (pos_info != None):   
            pos_td = pos_info.findAll('td')
            for td in pos_td:
                if(td.text.strip().isdigit()):
                    tempData.append(int(td.text.strip()))
                else:
                    tempData.append(td.text.strip())
        position_data.append(tempData)
    
    pos_df = pd.DataFrame(position_data)
    pos_df.fillna("Rookie", inplace=True)
    pos_df.columns = position_headers
    
    filename = position + '_stats.xlsx'
    pos_df.to_excel(filename)
    

OLB++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Chandler Jones
Terrell Suggs
Brooks Reed
Matt Longacre
Dante Booker
Pita Taumoepenu
De'Vondre Campbell
Foyesade Oluokun
Duke Riley
Chris Board
Matt Judon
Pernell McPhee
Nicholas Grigsby
Tyus Bowser
Tim Williams
Silas Stewart
Jaylon Ferguson
Shane Ray
Otaro Alaka
Donald Payne
Matt Milano
Lorenzo Alexander
Maurice Alexander
Corey Thompson
Vosean Joseph
Deon Lacey
Sam Acho
Mario Addison
Bruce Irvin
Brian Burns
Marquis Haynes
Christian Miller
Sione Teuhema
Khalil Mack
Leonard Floyd
Isaiah Irving
Aaron Lynch
James Vaughters
Kylie Fitts
Jordan Evans
Nick Vigil
Germaine Pratt
Deshaun Davis
Malik Jefferson
Curtis Akins
Sterling Sheffield
Genard Avery
Christian Kirksey
Mack Wilson
Adarius Taylor
Ray Ray
Willie Harvey
Anthony Stubbs
Leighton Vander
Sean Lee
Chris Covington
Justin March
Luke Gifford
Nate Hall
Bradley Chubb
Von Miller
Justin Hollins
Dekoda Watson
Malik Reed
Devon Kennard
Christian Jones
Jalen Ree